# Final Projet: Movie Pairing System
In order to use the data necessary, please download the `ml-1m`, `title.basics.tsv` and `title.crew.tsv` datasets, which can be done directly using the `data_download.sh` shell script provided, or by running the first cell of this notebook.
### Why those 3 datasets ?
The `ml-1m` dataset is chosen as it contains 1 million reviews for around 4 thousand movies, which makes for a good amount in order to recommend movies later down the line. the `title.basics.tsv` dataset contains an enormous amount of on movies, with more information that seen on the `ml-1m` dataset, but unfortunately does not contain reviews, and contains much more than movies, so it needs to be properly cleaned up. The `title.crew.tsv` dataset is chosen as a complimentary dataset, as it contains additional information for the movies, including the directors, which will be used as a feature in our model. At first, the `names.basic.tsv` dataset was used instead, but it was very soon clear that it would require more work to extract the same information, and could give false information as well (such an example being Clint Eastwood, which would be put as a director in movies he acted in). As such, to avoid mistakes and potential problems, it was dropped in favor of `title.crew.tsv`. While we do not have the actual names, this is not a problem for the model.

In [21]:
%%bash
if [ ! -d "data/movielens_latest" ];
then
    wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
    mkdir -p data/movielens_latest
    unzip -o ml-1m.zip -d data/movielens_latest;
else
    echo "Movielens data already downloaded";
fi

if [ ! -d "data/imdb" ];
then
    mkdir data/imdb
    wget https://datasets.imdbws.com/title.crew.tsv.gz
    gunzip -c title.crew.tsv.gz > data/imdb/title.crew.tsv;
    wget https://datasets.imdbws.com/title.basics.tsv.gz
    gunzip -c title.basics.tsv.gz > data/imdb/title.basics.tsv;
else
    echo "imdb data already downloaded";
fi

Movielens data already downloaded
imdb data already downloaded


In [23]:
# Imports for the project
import pandas as pd
import re
import random
import numpy as np
from sklearn.metrics import precision_score
from typing import Tuple, Union
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

## Data Analysis and Feature Engineering
In this section, we check the integrity of the downloaded files before fusing them and making some adjustments to them. First, we look at IMDB.

In [ ]:
df_crew_basics = pd.read_csv("data/imdb/title.crew.tsv", sep='\t')
df_title_basics =  pd.read_csv("data/imdb/title.basics.tsv", sep='\t', low_memory=False)
df_title_basics

We drop all titles that aren't movies from the titles dataset, merge both datasets on `tconst`, and keep only the `primaryTitle`,`startYear`, and `directors` columns.

In [6]:
df_title_basics = df_title_basics.drop(df_title_basics[df_title_basics['titleType'] != 'movie'].index)
df_imdb = pd.merge(df_title_basics, df_crew_basics, on='tconst', how='left')
df_imdb = df_imdb[['primaryTitle', 'startYear', 'directors']]
df_imdb

,primaryTitle,startYear,directors
0,Miss Jerry,1894,nm0085156
1,The Corbett-Fitzsimmons Fight,1897,nm0714557
2,Bohemios,1905,nm0063413
3,The Story of the Kelly Gang,1906,nm0846879
4,The Prodigal Son,1907,nm0141150
...,...,...,...
685588,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,"nm9272490,nm9272491"
685589,De la ilusión al desconcierto: cine colombiano...,2007,nm0652213
685590,Dankyavar Danka,2013,nm7764440
685591,6 Gunn,2017,nm10538612


We then look at the movielens dataset:

In [7]:
m_cols = ['MovieId', 'Title', 'Genres']
df_movielens = pd.read_csv('data/movielens_latest/ml-1m/movies.dat', sep='::', engine='python', encoding='latin-1', names=m_cols)
r_cols = ['UserId', 'MovieId', 'Rating', 'Timestamp']
df_ratings = pd.read_csv('data/movielens_latest/ml-1m/ratings.dat', sep='::', engine='python', encoding='latin-1', names=r_cols)
u_cols = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']
df_users = pd.read_csv('data/movielens_latest/ml-1m/users.dat', sep='::', engine='python', encoding='latin-1', names=u_cols)
df_movielens

,MovieId,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In order to have the same format as the IMDB files we want to clean up the titles by removing the date in the title and putting them in a separate column, then put the titles in lowercase. We'll put the titles in lowercase on the IMDB database later on as well.

In [8]:
def clean_titles(row):
    match = re.search(r'\((\d{4})\)$', row)
    if match:
        year = match.group(1)
        title = row[:match.start()].strip()
    else:
        year = None
        title = row
    if title.title().split(',')[-1].strip() in ['The', 'A']:
        title = (title.title().split(',')[-1].strip() + " " + " ".join(title.title().split(',')[:-1])).strip()

    title = title.lower()
    return title, year

df_movielens[['Title', 'Year']] = df_movielens['Title'].apply(clean_titles).apply(pd.Series)
df_movielens

,MovieId,Title,Genres,Year
0,1,toy story,Animation|Children's|Comedy,1995
1,2,jumanji,Adventure|Children's|Fantasy,1995
2,3,grumpier old men,Comedy|Romance,1995
3,4,waiting to exhale,Comedy|Drama,1995
4,5,father of the bride part ii,Comedy,1995
...,...,...,...,...
3878,3948,meet the parents,Comedy,2000
3879,3949,requiem for a dream,Drama,2000
3880,3950,tigerland,Drama,2000
3881,3951,two family house,Drama,2000


We separate the genres and make a one hot encoding on them as feature extraction:

In [9]:
df_movielens['Genres'] = df_movielens['Genres'].str.split('|')
df_movielens_exploded = df_movielens.explode('Genres')
df_movielens_ohe = pd.get_dummies(df_movielens_exploded['Genres'])
df_movielens_ohe_grouped = df_movielens_ohe.groupby(df_movielens_exploded.index).sum()
df_movielens_final = pd.concat([df_movielens.drop(columns=['Genres']), df_movielens_ohe_grouped], axis=1)
df_movielens_final

,MovieId,Title,Year,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,toy story,1995,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,jumanji,1995,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,grumpier old men,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,waiting to exhale,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,father of the bride part ii,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,meet the parents,2000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3879,3949,requiem for a dream,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3880,3950,tigerland,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3881,3951,two family house,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We merge the datasets in order to keep only the relevant movies before further working on the dataset. To do so, we decide to do the following:
- first, we merge both datasets on title and year
- since there are duplicates in the resulting dataset, we remove the duplicates
- we drop the `primaryTitle` and `startYear` columns
- we fill all NaN in the directors column with an empty line
- we one hot encode on the director to have a feature for each director

In [10]:
df_imdb['primaryTitle'] = df_imdb['primaryTitle'].str.lower()
df_movies = pd.merge(df_movielens_final, df_imdb, left_on=['Title', 'Year'], right_on=['primaryTitle', 'startYear'], how='left')
df_movies = df_movies.drop_duplicates(subset='MovieId', keep='first')
df_movies = df_movies.drop(columns=['primaryTitle','startYear'])
df_movies['directors'] = df_movies['directors'].fillna('')
df_movies

,MovieId,Title,Year,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,directors
0,1,toy story,1995,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,nm0005124
1,2,jumanji,1995,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,nm0002653
2,3,grumpier old men,1995,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,nm0222043
3,4,waiting to exhale,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,nm0001845
4,5,father of the bride part ii,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,nm0796124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3898,3948,meet the parents,2000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,nm0005366
3899,3949,requiem for a dream,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm0004716
3900,3950,tigerland,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm0001708
3901,3951,two family house,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm0208343


In [11]:
df_movies['directors'] = df_movies['directors'].str.split(',')
df_movies_exploded = df_movies.explode('directors')
df_movies_ohe = pd.get_dummies(df_movies_exploded['directors'])
df_movies_ohe_grouped = df_movies_ohe.groupby(df_movies_exploded.index).sum()
df_final = pd.concat([df_movies.drop(columns=['directors']), df_movies_ohe_grouped], axis=1)
df_final

,MovieId,Title,Year,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,nm1212046,nm1301264,nm15572750,nm15643153,nm1635724,nm1871737,nm1968474,nm1969399,nm2259353,nm9054338
0,1,toy story,1995,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,jumanji,1995,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,grumpier old men,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,waiting to exhale,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,father of the bride part ii,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3898,3948,meet the parents,2000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3899,3949,requiem for a dream,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3900,3950,tigerland,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3901,3951,two family house,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now that we have a proper dataset with a reasonable amount of features, time to add the ratings to the mix. We will create a user-item matrix that will be used by our model. We also normalize the ratings afterwards, once we've replaced all NaN with 0s.

In [12]:
df_matrix = df_ratings.pivot(index="UserId", columns="MovieId", values="Rating")
df_matrix

MovieId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserId,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Building the model
We decide to use collaborative filtering alongside a "couple rating", which is the mean of scores of both people on a single movie. The goal is to maximize that score when recommending a movie. We decide to make the following choice: The users are already in the database, and the user simply give the id of the users that want a recommendation. This is because using another completely different person with its own recommendations would not come out as the predictions does not take them into account.

We reuse some functions seen during the lessons and available on the utils section of the professor's github page.

In [13]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y) * R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_ / 2) * (
        tf.reduce_sum(X**2) + tf.reduce_sum(W**2) + tf.reduce_sum(W**2)
    )
    return J

def normalizeRatings(Y: np.ndarray, R: np.ndarray, axis: int = 1) -> Tuple:
    Ymean = (np.sum(Y * R, axis=axis) / (np.sum(R, axis=axis) + 1e-12)).reshape(-1, 1)

    if axis == 0:
        Ynorm = Y.T - np.multiply(Ymean, R.T)
    else:
        Ynorm = Y - np.multiply(Ymean, R)

    return (Ynorm, Ymean)

def split_ratings(
    ratings: np.ndarray, val_size: Union[float, int]
) -> Tuple[np.ndarray, np.ndarray]:
    rows, cols = np.where(ratings > 0)
    val_ratings = np.zeros(ratings.shape)
    train_ratings = ratings.copy()

    if val_size < 1:  # Hence it is a percentage
        val_size = int(rows * val_size)

    val_ids = random.sample(range(len(rows)), val_size)

    for row, col in zip(rows[val_ids], cols[val_ids]):
        train_ratings[row, col] = 0
        val_ratings[row, col] = ratings[row, col]

    return train_ratings, val_ratings

In [14]:
R = df_matrix.notna().astype(int).to_numpy()
df_matrix = df_matrix.fillna(0)
Y = df_matrix.to_numpy()
num_movies, num_users = Y.shape
df_features = df_final.drop(columns=['Title', 'Year'])
num_features = len(df_features.columns)
moviesList = df_final['Title'].to_list()

Y_train, Y_val = split_ratings(Y, val_size=627)
Y_train, Y_test = split_ratings(Y_train, val_size=627)
Ynorm, Ymean = normalizeRatings(Y_train, R)

We decide to evaluate the prediction using RMSE and the precision, as both can give accurate reading because RMSE is very sensitive to big gaps in data, and the precision is simply a way to get an idea of the overall "correctness" of the model.

In [16]:
class Recommender:
    def __init__(self, num_users, num_movies, num_features, features):
        self.num_users = num_users
        self.num_movies = num_movies
        self.num_features = num_features
        self.features = features

        self.mean = None

        tf.random.set_seed(1234)  # for consistent results
        self.W = tf.Variable(tf.random.normal((self.num_users, self.num_features), dtype=tf.float64), name="W")
        self.X = tf.Variable(tf.random.normal((self.num_movies, self.num_features), dtype=tf.float64), name="X")
        self.b = tf.Variable(tf.random.normal((1, self.num_users), dtype=tf.float64), name="b")



    def fit(self, Y, R, iterations):
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-1)
        lambda_ = 1
        for iter in range(iterations):
            with tf.GradientTape() as tape:
                cost_value = cofi_cost_func_v(self.X, self.W, self.b, Y, R, 1)

            grads = tape.gradient(cost_value, [self.X, self.W, self.b])

            optimizer.apply_gradients(zip(grads, [self.X,self.W, self.b]))
            if iter % 20 == 0:
                print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

    def predict(self):
        p = np.matmul(self.X.numpy(), np.transpose(self.W.numpy())) + self.b.numpy()
        pm = p + self.mean

        return pm

    def check_metrics(self, Y):
        pm = self.predict()

        rows, cols = Y.shape()

        predicted_ratings = pm[rows, cols]
        real_ratings = Y[rows, cols]
        rmse = np.sqrt(np.mean((predicted_ratings - real_ratings) ** 2))

        # We consider 3 to be an average grade, so all pertinent predictions are above 3
        predicted_pertinent = predicted_ratings > 3
        real_pertinent = real_ratings > 3

        precision = precision_score(real_pertinent, predicted_pertinent)

        return precision, rmse

    def recommend_movie(self, user_1, user_2):

        pm = self.predict()

        user_1_ratings = pm[:, user_1]
        user_2_ratings = pm[:, user_2]

        couple_ratings = (user_1_ratings + user_2_ratings) / 2

        couple_scores = []
        for i in range(len(couple_ratings)):
            couple_scores[i] = couple_ratings[i]

        best_choice = max(couple_scores, key=lambda x: x[1])

        return best_choice[0]

In [18]:
recommender = Recommender(num_users, num_movies, num_features, df_features)
recommender.fit(Ynorm, Ymean, R *(Y_train > 0), 400)

NameError: name 'tf' is not defined

In [24]:
movie = recommender.recommend_movie(1, 2)
movie

NameError: name 'recommender' is not defined